In [1]:
import json, os
from nltk import tokenize

In [2]:
working_dir = 'flan-t5-prompted/questions/'
answer_files = os.listdir(working_dir)
answer_files = answer_files[1:]

In [3]:
answers_per_file = []
all_answers = []

for f in answer_files:
    with open(working_dir + f) as f:
        data = json.load(f)
    print(len(data['question']))
    answers_per_file.append(data['question'])
    all_answers += data['question']

50
50


In [4]:
all_answers

[{'fluency': 'A. Yes',
  '_id': 'main_issue_0',
  'uid': '3261712960.jpg#0r1n',
  'generation_label': 'entailment',
  'explanation_only': 'A. Entailment',
  'explanation_with_premise': 'A. Entailment',
  'consistency': 'NO'},
 {'fluency': 'A. Yes',
  '_id': 'main_issue_1',
  'uid': '3261712960.jpg#0r1c',
  'generation_label': 'neutral',
  'explanation_only': 'C. Neutral',
  'explanation_with_premise': 'C. Neutral',
  'consistency': 'NO'},
 {'fluency': 'A. Yes',
  '_id': 'main_issue_2',
  'uid': 'ibm_1',
  'generation_label': 'entailment',
  'explanation_only': 'B. Contradiction',
  'explanation_with_premise': 'C. Neutral',
  'consistency': 'YES'},
 {'fluency': 'A. Yes',
  '_id': 'main_issue_3',
  'uid': '202029970.jpg#1r1e',
  'generation_label': 'contradiction',
  'explanation_only': 'B. Contradiction',
  'explanation_with_premise': 'A. Entailment',
  'consistency': 'YES'},
 {'fluency': 'A. Yes',
  '_id': 'main_issue_4',
  'uid': '5964395383.jpg#1r1n',
  'generation_label': 'neutral',

In [5]:
for q in all_answers:
    
    try:
        if q['fluency'] == 'A. Yes':
            q['fluency'] = True
        elif q['fluency'] == 'B. No':
            q['fluency'] = False
        
        if 'explanation_only' in q.keys():
            if q['explanation_only'] == 'A. Entailment':
                q['explanation_only'] = 'entailment'
            elif q['explanation_only'] == 'B. Contradiction':
                q['explanation_only'] = 'contradiction'
            elif q['explanation_only'] == 'C. Neutral':
                q['explanation_only'] = 'neutral'
            elif q['explanation_only'] == 'D. Not Sure':
                q['explanation_only'] = 'Not Sure'

            if q['explanation_with_premise'] == 'A. Entailment':
                q['explanation_with_premise'] = 'entailment'
            elif q['explanation_with_premise'] == 'B. Contradiction':
                q['explanation_with_premise'] = 'contradiction'
            elif q['explanation_with_premise'] == 'C. Neutral':
                q['explanation_with_premise'] = 'neutral'
            elif q['explanation_with_premise'] == 'D. Not Sure':
                q['explanation_with_premise'] = 'Not Sure'

            if q['consistency'] == 'YES':
                q['consistency'] = True
            else:
                q['consistency'] = False
        else:
            q['explanation_only'] = None
            q['explanation_with_premise'] = None
            q['consistency'] = None
    except:
        print(q)
        q['consistency'] = False
        pass

In [6]:
with open('flan-t5-prompted/flan-t5-prompted_90_snli_10_ibm.json', 'r') as f:
    original_data = json.load(f)

for i in original_data:
    for j in all_answers:
        if i['uid'] == j['uid'] and i['generation_label'] == j['generation_label']:
            for k in j.keys():
                i[k] = j[k]


with open('data/chaosNLI/chaosNLI_snli.json', 'r') as f:
    chaoSNLI = json.load(f)

for i in chaoSNLI:
    for j in original_data:
        if i['uid'] == j['uid']:
            j['majority_label'] = i['majority_label']
            j['label_counter'] = i['label_counter']
            labels = ['c', 'e', 'n']
            for l in labels:
                if not(l in j['label_counter']):
                    j['label_counter'][l] = 0 
            ranked_labels = sorted([(x,j['label_counter'][x]) for x in labels], key=lambda x : x[1], reverse=True) # higher first
            j['label_rank'] = {ranked_labels[0][0]:'high', ranked_labels[1][0]:'mid', ranked_labels[2][0]:'low'}
            
with open('flan-t5-prompted/processed.json', 'w') as f:
    f.write(json.dumps(original_data, indent=2))
    f.close()

In [10]:
fluency = 0
counter = 0
res = {'high':0, 'mid':0, 'low':0}
label_counter = {'high':0, 'mid':0, 'low':0}
for q in original_data:
    if not('ibm' in q['uid']):
        counter +=1
        rank = q['label_rank'][q['generation_label'][0]]
        label_counter[rank] += 1
        if q['generation_label'] == q['explanation_only'] and q['explanation_with_premise'] and q['consistency'] ==True:
            fluency += 1
            res[rank] += 1
            print(q)
for key in res.keys():
    res[key] = res[key] / label_counter[key]

print(res)

{'premise': 'A young boy with black hair is holding onto a gun which is secured to a vehicle which is being driven by a young man who is wearing a bright pink shirt.', 'hypothesis': 'The bank has just been robbed.', 'explanation': 'A bank is not a vehicle.', 'generation_label': 'contradiction', 'uid': '4762813376.jpg#0r1c', 'fluency': True, '_id': 'main_issue_7', 'explanation_only': 'contradiction', 'explanation_with_premise': 'contradiction', 'consistency': True, 'majority_label': 'n', 'label_counter': {'n': 86, 'c': 12, 'e': 2}, 'label_rank': {'n': 'high', 'c': 'mid', 'e': 'low'}}
{'premise': 'A young boy with black hair is holding onto a gun which is secured to a vehicle which is being driven by a young man who is wearing a bright pink shirt.', 'hypothesis': 'The boy is outside and running from cops.', 'explanation': 'A young boy is running from cops implies a young man with black hair is holding onto a gun which is secured to a vehicle which is being driven by a young man who is we

In [16]:
fluency = 0
counter = 0
res = {'high':0, 'mid':0, 'low':0}
label_counter = {'high':0, 'mid':0, 'low':0}
for q in original_data:
    if not('ibm' in q['uid']):
        counter +=1
        rank = q['label_rank'][q['generation_label'][0]]
        label_counter[rank] += 1
        if q['fluency'] == True and q['consistency'] == False:
            fluency += 1
            res[rank] += 1
        else:
            print(q)
for key in res.keys():
    res[key] = res[key] / label_counter[key]

print(res)

{'premise': 'A man in a blue shirt standing in front of a garage-like structure painted with geometric designs.', 'hypothesis': 'A man is wearing a blue shirt', 'explanation': 'A man in a blue shirt is not necessarily wearing a blue shirt.', 'generation_label': 'neutral', 'uid': '24467676.jpg#1r1e', 'fluency': True, '_id': 'main_issue_0', 'explanation_only': 'neutral', 'explanation_with_premise': 'entailment', 'consistency': True, 'majority_label': 'e', 'label_counter': {'e': 98, 'c': 2, 'n': 0}, 'label_rank': {'e': 'high', 'c': 'mid', 'n': 'low'}}
{'premise': 'At an outdoor event in an Asian-themed area, a crowd congregates as one person in a yellow Chinese dragon costume confronts the camera.', 'hypothesis': 'A crowd is in front of a camera', 'explanation': 'Congregates does not imply being in front of.', 'generation_label': 'neutral', 'uid': '3261712960.jpg#0r1e', 'fluency': True, '_id': 'main_issue_2', 'explanation_only': 'neutral', 'explanation_with_premise': 'entailment', 'consis

In [18]:
fluency = 0
counter = 0
res = {'high':0, 'mid':0, 'low':0}
label_counter = {'high':0, 'mid':0, 'low':0}
for q in original_data:
    if not('ibm' in q['uid']):
        counter +=1
        rank = q['label_rank'][q['generation_label'][0]]
        label_counter[rank] += 1
        if q['explanation_only'] == q['generation_label']:
            fluency += 1
            res[rank] += 1
        else:
            print(q)
for key in res.keys():
    res[key] = res[key] / label_counter[key]

print(res)

{'premise': 'A baby at the end of a slip and slide at a party', 'hypothesis': 'The baby is wet.', 'explanation': 'A baby at the end of a slip and slide at a party implies that the baby is wet..so the baby is wet..so the baby is at a party..so the baby is wet..so the baby is at a party..so the baby is wet..so the baby is at a party..so the baby is wet..so the baby is at a party..so the baby is wet..so the baby is at a party.', 'generation_label': 'entailment', 'uid': '2537197415.jpg#3r1n', 'fluency': False, '_id': 'main_issue_9', 'explanation_only': None, 'explanation_with_premise': None, 'consistency': None, 'majority_label': 'n', 'label_counter': {'e': 42, 'n': 58, 'c': 0}, 'label_rank': {'n': 'high', 'e': 'mid', 'c': 'low'}}
{'premise': 'At an outdoor event in an Asian-themed area, a crowd congregates as one person in a yellow Chinese dragon costume confronts the camera.', 'hypothesis': 'A crowd is in front of a camera', 'explanation': 'A crowd congregates as one person confronts the

In [20]:
fluency = 0
counter = 0
res = {'high':0, 'mid':0, 'low':0}
label_counter = {'high':0, 'mid':0, 'low':0}
for q in original_data:
    if not('ibm' in q['uid']):
        counter +=1
        rank = q['label_rank'][q['generation_label'][0]]
        label_counter[rank] += 1
        if q['explanation_only'] == q['generation_label'] and q['explanation_only'] == q['explanation_with_premise'] and (q['fluency'] == True and q['consistency'] == False):
            fluency += 1
            res[rank] += 1
        else:
            print(q)
for key in res.keys():
    res[key] = res[key] / label_counter[key]

print(res)

{'premise': 'A man in a blue shirt standing in front of a garage-like structure painted with geometric designs.', 'hypothesis': 'A man is wearing a blue shirt', 'explanation': 'A man in a blue shirt is not necessarily wearing a blue shirt.', 'generation_label': 'neutral', 'uid': '24467676.jpg#1r1e', 'fluency': True, '_id': 'main_issue_0', 'explanation_only': 'neutral', 'explanation_with_premise': 'entailment', 'consistency': True, 'majority_label': 'e', 'label_counter': {'e': 98, 'c': 2, 'n': 0}, 'label_rank': {'e': 'high', 'c': 'mid', 'n': 'low'}}
{'premise': 'At an outdoor event in an Asian-themed area, a crowd congregates as one person in a yellow Chinese dragon costume confronts the camera.', 'hypothesis': 'A crowd is in front of a camera', 'explanation': 'Congregates does not imply being in front of.', 'generation_label': 'neutral', 'uid': '3261712960.jpg#0r1e', 'fluency': True, '_id': 'main_issue_2', 'explanation_only': 'neutral', 'explanation_with_premise': 'entailment', 'consis

In [21]:
fluency = 0
counter = 0
for q in original_data:
    if 'ibm' in q['uid']:
        counter += 1
        if q['fluency'] == True:
            fluency += 1
print(fluency/counter)

0.8


In [22]:
fluency = 0
counter = 0
for q in original_data:
    if 'ibm' in q['uid']:
        counter += 1
        if q['fluency'] == True and q['consistency'] == False:
            fluency += 1
print(fluency/counter)

0.6


In [23]:
fluency = 0
counter = 0
for q in original_data:
    if 'ibm' in q['uid']:
        counter += 1
        if q['generation_label'] == q['explanation_only']:
            fluency += 1
print(fluency/counter)

0.5


In [24]:
ibm_persuasive = 0
counter = 0
for q in original_data:
    if 'ibm' in q['uid']:
        counter += 1
        if q['generation_label'] == q['explanation_only'] and q['explanation_only'] == q['explanation_with_premise']:
            ibm_persuasive += 1
print(ibm_persuasive/counter)

0.4


In [19]:
ibm_persuasive = 0
counter = 0
for q in original_data:
    if 'ibm' in q['uid']:
        counter += 1
        if q['fluency'] == True and q['consistency'] == False:
            ibm_persuasive += 1
print(ibm_persuasive/counter)

0.9
